In [ ]:
#Import of data as CSV
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from google.colab import drive

In [7]:
df = pd.read_csv('passenger_data.csv',encoding = "utf-8")

In [8]:
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [9]:
# Drop the column Nr 1
df = df.drop(df.columns[[0]], axis=1)

In [11]:
# Capitalise the column names 'id' and 'satisfaction'
df.rename(columns={'id': 'ID', 'satisfaction': 'Satisfaction'}, inplace=True)

In [12]:
# Capitalise 'disloyal' in the column Nr 3 and "travel" in column Nr 5
df['Customer Type'] = df['Customer Type'].str.replace('disloyal', 'Disloyal')
df['Type of Travel']= df['Type of Travel'].str.replace ('Business travel', 'Business Travel' )

In [13]:
# Show null values (true= no value)
nullvalues = df.isnull()
print(nullvalues)

           ID  Gender  Customer Type    Age  Type of Travel  Class  \
0       False   False          False  False           False  False   
1       False   False          False  False           False  False   
2       False   False          False  False           False  False   
3       False   False          False  False           False  False   
4       False   False          False  False           False  False   
...       ...     ...            ...    ...             ...    ...   
103919  False   False          False  False           False  False   
103920  False   False          False  False           False  False   
103921  False   False          False  False           False  False   
103922  False   False          False  False           False  False   
103923  False   False          False  False           False  False   

        Flight Distance  Inflight wifi service  \
0                 False                  False   
1                 False                  False   
2        

In [14]:
# Count of how many null values there are in each column
nullvalues = df.isnull().sum()
print(nullvalues)

ID                                     0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
Satisfaction                           0
dtype: int64


# The data is complete for all columns except for "Arrival delay in minutes" where 310 out of 103923 values are missing. As this does not represent a very large number and information on arrival delay is probably important to assess customer satisfaction, we decided to drop the rows containing null values and to only use complete data for our model.

In [15]:
# Drop null values
df_new = df.copy()
df_new = df.dropna()

In [16]:
# New data frame (df_new) has no null values
df_new.isnull().sum()

ID                                   0
Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Inflight wifi service                0
Departure/Arrival time convenient    0
Ease of Online booking               0
Gate location                        0
Food and drink                       0
Online boarding                      0
Seat comfort                         0
Inflight entertainment               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Inflight service                     0
Cleanliness                          0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
Satisfaction                         0
dtype: int64

In [17]:
# Transform column "Arrival Delay in Minutes" in integer numbers
df = df.fillna(0)
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].astype(int)

In [18]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103614 entries, 0 to 103923
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ID                                 103614 non-null  int64  
 1   Gender                             103614 non-null  object 
 2   Customer Type                      103614 non-null  object 
 3   Age                                103614 non-null  int64  
 4   Type of Travel                     103614 non-null  object 
 5   Class                              103614 non-null  object 
 6   Flight Distance                    103614 non-null  int64  
 7   Inflight wifi service              103614 non-null  int64  
 8   Departure/Arrival time convenient  103614 non-null  int64  
 9   Ease of Online booking             103614 non-null  int64  
 10  Gate location                      103614 non-null  int64  
 11  Food and drink                     1036

In [19]:
df_new.describe()

,ID,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
count,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000,103614.000000
mean,64943.326336,39.379794,1189.278736,2.729786,3.060137,2.756992,2.977020,3.202135,3.250478,3.439690,3.358282,3.382632,3.351304,3.631691,3.304264,3.640772,3.286293,14.745826,15.176414
std,37460.794345,15.113194,997.272823,1.327819,1.525226,1.398941,1.277739,1.329352,1.349420,1.318842,1.333006,1.288292,1.315464,1.181087,1.265411,1.175622,1.312190,38.113593,38.695437
min,1.000000,7.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32562.250000,27.000000,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000
50%,64894.500000,40.000000,842.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000
75%,97370.500000,51.000000,1742.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,12.000000,13.000000
max,129880.000000,85.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1592.000000,1584.000000


In [ ]:
##Data Analysis
Let's have a look at how many customers are satisfied (or not).

In [20]:
df_new.groupby('Satisfaction').count()

,ID,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
Satisfaction,,,,,,,,,,,,,,,,,,,,,
neutral or dissatisfied,58708,58708,58708,58708,58708,58708,58708,58708,58708,58708,...,58708,58708,58708,58708,58708,58708,58708,58708,58708,58708
no answer,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
satisfied,44900,44900,44900,44900,44900,44900,44900,44900,44900,44900,...,44900,44900,44900,44900,44900,44900,44900,44900,44900,44900


In [21]:
##And how many customers are loyal customers.
df_new.groupby('Customer Type').count()

,ID,Gender,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Satisfaction
Customer Type,,,,,,,,,,,,,,,,,,,,,
Disloyal Customer,18939,18939,18939,18939,18939,18939,18939,18939,18939,18939,...,18939,18939,18939,18939,18939,18939,18939,18939,18939,18939
Loyal Customer,84675,84675,84675,84675,84675,84675,84675,84675,84675,84675,...,84675,84675,84675,84675,84675,84675,84675,84675,84675,84675


In [22]:
%matplotlib inline

In [23]:
##Fin out the age groups
df['Age']

0         13
1         25
2         26
3         25
4         61
          ..
103919    34
103920    23
103921    17
103922    14
103923    42
Name: Age, Length: 103924, dtype: int64

In [24]:
#Count the number of rows with different Ages
df.groupby('Age').count()

,ID,Gender,Customer Type,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Satisfaction
Age,,,,,,,,,,,,,,,,,,,,,
7,562,562,562,562,562,562,562,562,562,562,...,562,562,562,562,562,562,562,562,562,562
8,640,640,640,640,640,640,640,640,640,640,...,640,640,640,640,640,640,640,640,640,640
9,692,692,692,692,692,692,692,692,692,692,...,692,692,692,692,692,692,692,692,692,692
10,683,683,683,683,683,683,683,683,683,683,...,683,683,683,683,683,683,683,683,683,683
11,678,678,678,678,678,678,678,678,678,678,...,678,678,678,678,678,678,678,678,678,678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,87,87,87,87,87,87,87,87,87,87,...,87,87,87,87,87,87,87,87,87,87
78,33,33,33,33,33,33,33,33,33,33,...,33,33,33,33,33,33,33,33,33,33
79,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42


In [25]:
#The avarage age
np.mean( df['Age'] )

39.37903660367191

In [26]:
#Count the number of rows with different genders
df_new.groupby('Gender').count()

,ID,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Satisfaction
Gender,,,,,,,,,,,,,,,,,,,,,
Diverse,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
Female,52577,52577,52577,52577,52577,52577,52577,52577,52577,52577,...,52577,52577,52577,52577,52577,52577,52577,52577,52577,52577
Male,51021,51021,51021,51021,51021,51021,51021,51021,51021,51021,...,51021,51021,51021,51021,51021,51021,51021,51021,51021,51021
na,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9


In [27]:
#Check-out if there are duplicates
df.duplicated(subset=None, keep='first')

0         False
1         False
2         False
3         False
4         False
          ...  
103919    False
103920    False
103921    False
103922    False
103923    False
Length: 103924, dtype: bool

In [ ]:
##Find out Data outliers